In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
nights = pd.read_csv('n_per_accommodation.csv')
travellers = pd.read_csv('t_per_accommodation.csv')
stay = pd.read_csv('stay.csv')
xunta = pd.read_csv('data_from_xunta.csv')

In [3]:
def drop0(df):
    df = df.drop(['Unnamed: 0'],axis=1)
    return df

In [4]:
nights = drop0(nights)
travellers = drop0(travellers)
stay = drop0(stay)
xunta = drop0(xunta)

In [5]:
print(nights.shape)
print(travellers.shape)
print(stay.shape)
print(xunta.shape)

(2288, 5)
(2288, 5)
(1144, 4)
(3120, 10)


In [6]:
def clean_1(df):
    # .. not available data: I will drop this, it's from earlier years and it's data we don't know
    # . protected data: I will fill it now with -1 to try to work with it later when all the data is together
    df['total'] = np.where(df['total'].isin(['.']),-1,df['total'])
    df = df[df['total'] != '..']
    
    # I will fill NaN with 0
    nights[nights['total'].isna()]
    
    # I want to change nights and travellers: get the column 'come_from' to be converted in total_Spain and total_Foreign
    df['Spain'] = np.where(df['come_from'].isin(['Spain']), df['total'], 0)
    df['Foreign'] = np.where(df['come_from'].isin(['Foreign']), df['total'],0)
    
    #now I don't need these 2 columns
    df = df.drop(['come_from','total'],axis=1)
    
    #I will fill NaN values with 0 because there are 4 of them in moments that may be no activity
    df = df.fillna(0)
    
    # Now I'll change the type of the new columns to number
    df['Spain'] = df['Spain'].astype(str).astype(int)
    df['Foreign'] = df['Foreign'].astype(str).astype(int)
    
    
    # I will split the data now to work on it better
    df_1 = df[(df['Spain'].isin([-1]))|(df['Foreign'].isin([-1]))]
    df_2 = df[(df['Spain'] != -1)&(df['Foreign'] != -1)]
    
    # Now I sort it so I have the info from Spain and from Foreign in the same row, for the same period and accommodation
    df_2 = df_2.groupby(['accommodation_type','year','month']).agg({'Spain':max,'Foreign':sum}).reset_index()
    df_1 = df_1.groupby(['accommodation_type','year','month']).agg({'Spain':min,'Foreign':min}).reset_index()
    
    #And finally merge the 2 df to get all the information together and deal with -1 values
    new_df = pd.concat([df_2,df_1],axis=0)
    
    return new_df

In [7]:
nights_clean = clean_1(nights)
travellers_clean = clean_1(travellers)

C:\Users\Julia\AppData\Local\Temp\ipykernel_29508\2758756134.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Spain'] = np.where(df['come_from'].isin(['Spain']), df['total'], 0)
C:\Users\Julia\AppData\Local\Temp\ipykernel_29508\2758756134.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Foreign'] = np.where(df['come_from'].isin(['Foreign']), df['total'],0)
C:\Users\Julia\AppData\Local\Temp\ipykernel_29508\2758756134.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [8]:
travellers_clean

,accommodation_type,year,month,Spain,Foreign
0,Albergue,2014,Apr,166,183
1,Albergue,2014,Aug,3724,617
2,Albergue,2014,Dec,153,1
3,Albergue,2014,Feb,43,11
4,Albergue,2014,Jan,27,10
...,...,...,...,...,...
25,Campings,2020,Dec,-1,-1
26,Campings,2020,May,-1,-1
27,Rural Tourism,2020,Apr,-1,-1
28,Rural Tourism,2020,May,-1,-1


In [9]:
merge = pd.merge(left = travellers_clean,
        right = nights_clean,
        how = 'inner', 
        left_on = 'accommodation_type',
        right_on= 'accommodation_type')
                                                    

In [10]:
merge = merge[(merge['year_x']==merge['year_y'])&(merge['month_x']==merge['month_y'])]

In [11]:
merge = merge.drop(['year_y','month_y'],axis=1)
merge.columns = ['accommodation_type','year','month','travellers_Spain','travellers_Foreign','nights_Spain','nights_Foreign']

In [12]:
merge

,accommodation_type,year,month,travellers_Spain,travellers_Foreign,nights_Spain,nights_Foreign
0,Albergue,2014,Apr,166,183,1369,1366
105,Albergue,2014,Aug,3724,617,13270,898
210,Albergue,2014,Dec,153,1,179,1
315,Albergue,2014,Feb,43,11,74,11
420,Albergue,2014,Jan,27,10,43,16
...,...,...,...,...,...,...,...
279652,Rural Tourism,2022,Mar,4582,665,6821,925
279913,Rural Tourism,2022,May,12934,9716,19445,11792
280174,Rural Tourism,2020,Apr,-1,-1,-1,-1
280435,Rural Tourism,2020,May,-1,-1,-1,-1


In [13]:
stay['avg_days'].value_counts()

.       27
2       16
1.98    15
2.03    14
1.75    13
        ..
3.45     1
3.53     1
6.57     1
5.8      1
7.84     1
Name: avg_days, Length: 363, dtype: int64

In [14]:
stay[stay['avg_days'] == '..']

,accommodation_type,avg_days,year,month
486,Campings,..,2003,Oct


In [15]:
merge2 = pd.merge(left = stay,
        right = merge,
        how = 'left', 
        left_on = 'accommodation_type',
        right_on= 'accommodation_type')

In [16]:
merge3 = merge2[(merge2['year_x']==merge2['year_y'])&(merge2['month_x']==merge2['month_y'])]

In [17]:
merge3[merge3['avg_days'] == '..']

,accommodation_type,avg_days,year_x,month_x,year_y,month_y,travellers_Spain,travellers_Foreign,nights_Spain,nights_Foreign


In [18]:
# creating new columns for the total travellers and nights
merge3['total_travellers'] = merge3['travellers_Spain']+merge3['travellers_Foreign']
merge3['total_nights'] = merge3['nights_Spain']+merge3['nights_Foreign']

#filling . in avg_days column nights/travellers
merge3['avg_days'] = np.where(merge3['avg_days'].isin(['.']),merge3['total_nights']/merge3['total_travellers'],merge3['avg_days'])

#dropping inneccessary columns
merge3 = merge3.drop(['year_y','month_y'],axis=1)

#renaming and reordering columns
merge3.columns = ['accommodation_type','avg_days','year','month','travellers_Spain','travellers_Foreign','nights_Spain','nights_Foreign','total_travellers','total_nights']
merge3 = merge3[['accommodation_type','year','month','travellers_Spain','travellers_Foreign','total_travellers','nights_Spain','nights_Foreign','total_nights','avg_days']]

#changin type to number in avg_days column
merge3['avg_days'] = merge3['avg_days'].astype(np.number)

C:\Users\Julia\AppData\Local\Temp\ipykernel_29508\2307117083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge3['total_travellers'] = merge3['travellers_Spain']+merge3['travellers_Foreign']
C:\Users\Julia\AppData\Local\Temp\ipykernel_29508\2307117083.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge3['total_nights'] = merge3['nights_Spain']+merge3['nights_Foreign']
C:\Users\Julia\AppData\Local\Temp\ipykernel_29508\2307117083.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [19]:
merge3.dtypes

accommodation_type     object
year                    int64
month                  object
travellers_Spain        int32
travellers_Foreign      int32
total_travellers        int32
nights_Spain            int32
nights_Foreign          int32
total_nights            int32
avg_days              float64
dtype: object

In [20]:
xunta

,accommodation_type,spain_travelers,spain_nights,spain_avg_stay,galicia_travelers,galicia_nights,galicia_avg_stay,come_from,year,month
0,Hotels,3004915,10132886,3.4,112149,243320,2.2,All,2001,Jan
1,Hotels,3518883,11753586,3.3,131003,269836,2.1,All,2001,Feb
2,Hotels,4333254,14516657,3.4,153081,333034,2.2,All,2001,Mar
3,Hotels,5384678,18085751,3.4,192698,432250,2.2,All,2001,Apr
4,Hotels,5673119,21063744,3.7,190637,415208,2.2,All,2001,May
...,...,...,...,...,...,...,...,...,...,...
3115,Rural Tourism,71714,232823,..,5647,7273,..,Foreign,2022,Apr
3116,Rural Tourism,96813,288450,..,9716,11792,..,Foreign,2022,May
3117,Rural Tourism,100858,330551,..,7503,9626,..,Foreign,2022,Jun
3118,Rural Tourism,122802,496136,..,8461,16004,..,Foreign,2022,Jul


In [21]:
xunta_to_merge = xunta.drop(['galicia_travelers','galicia_nights','galicia_avg_stay'],axis=1)
xunta_to_merge

,accommodation_type,spain_travelers,spain_nights,spain_avg_stay,come_from,year,month
0,Hotels,3004915,10132886,3.4,All,2001,Jan
1,Hotels,3518883,11753586,3.3,All,2001,Feb
2,Hotels,4333254,14516657,3.4,All,2001,Mar
3,Hotels,5384678,18085751,3.4,All,2001,Apr
4,Hotels,5673119,21063744,3.7,All,2001,May
...,...,...,...,...,...,...,...
3115,Rural Tourism,71714,232823,..,Foreign,2022,Apr
3116,Rural Tourism,96813,288450,..,Foreign,2022,May
3117,Rural Tourism,100858,330551,..,Foreign,2022,Jun
3118,Rural Tourism,122802,496136,..,Foreign,2022,Jul


In [22]:
xunta_to_merge.isna().sum()

accommodation_type    0
spain_travelers       0
spain_nights          0
spain_avg_stay        0
come_from             0
year                  0
month                 0
dtype: int64

In [23]:
xunta_all = xunta_to_merge[xunta_to_merge['come_from'].isin(['All'])]
xunta_Spain = xunta_to_merge[xunta_to_merge['come_from'].isin(['Spain'])]
xunta_Foreign = xunta_to_merge[xunta_to_merge['come_from'].isin(['Foreign'])]

In [24]:
xunta_all = xunta_all.drop(['come_from'],axis=1)
xunta_all.columns = ['accommodation_type', 'total_spain_travellers', 'total_spain_nights', 'avg_spain_avg_stay','year','month']

In [25]:
xunta_Spain = xunta_Spain.drop(['spain_avg_stay','come_from'],axis=1)
xunta_Foreign = xunta_Foreign.drop(['spain_avg_stay','come_from'],axis=1)

In [26]:
xunta_Spain.columns = ['accommodation_type', 'spain_travellers_Spain', 'spain_nights_Spain','year','month']
xunta_Foreign.columns = ['accommodation_type', 'spain_travellers_Foreign', 'spain_nights_Foreign','year','month']

In [27]:
pd.merge(left = xunta_Spain,
        right = xunta_all,
        how = 'inner', 
        left_on = 'accommodation_type',
        right_on= 'accommodation_type')

,accommodation_type,spain_travellers_Spain,spain_nights_Spain,year_x,month_x,total_spain_travellers,total_spain_nights,avg_spain_avg_stay,year_y,month_y
0,Hotels,1876469,4371423,2001,Jan,3004915,10132886,3.4,2001,Jan
1,Hotels,1876469,4371423,2001,Jan,3518883,11753586,3.3,2001,Feb
2,Hotels,1876469,4371423,2001,Jan,4333254,14516657,3.4,2001,Mar
3,Hotels,1876469,4371423,2001,Jan,5384678,18085751,3.4,2001,Apr
4,Hotels,1876469,4371423,2001,Jan,5673119,21063744,3.7,2001,May
...,...,...,...,...,...,...,...,...,...,...
270395,Rural Tourism,504888,1866009,2022,Aug,413073,1076089,2.6,2022,Apr
270396,Rural Tourism,504888,1866009,2022,Aug,366923,853724,2.3,2022,May
270397,Rural Tourism,504888,1866009,2022,Aug,409495,1029162,2.5,2022,Jun
270398,Rural Tourism,504888,1866009,2022,Aug,553793,1792243,3.2,2022,Jul


I think i'm not merging this with my previuos, it's nice how it is, because it has not Albergues, so it gives me a lot of nans

In [28]:
xunta

,accommodation_type,spain_travelers,spain_nights,spain_avg_stay,galicia_travelers,galicia_nights,galicia_avg_stay,come_from,year,month
0,Hotels,3004915,10132886,3.4,112149,243320,2.2,All,2001,Jan
1,Hotels,3518883,11753586,3.3,131003,269836,2.1,All,2001,Feb
2,Hotels,4333254,14516657,3.4,153081,333034,2.2,All,2001,Mar
3,Hotels,5384678,18085751,3.4,192698,432250,2.2,All,2001,Apr
4,Hotels,5673119,21063744,3.7,190637,415208,2.2,All,2001,May
...,...,...,...,...,...,...,...,...,...,...
3115,Rural Tourism,71714,232823,..,5647,7273,..,Foreign,2022,Apr
3116,Rural Tourism,96813,288450,..,9716,11792,..,Foreign,2022,May
3117,Rural Tourism,100858,330551,..,7503,9626,..,Foreign,2022,Jun
3118,Rural Tourism,122802,496136,..,8461,16004,..,Foreign,2022,Jul


In [29]:
# cleanning the .. inside spain_travellers and nights, it's the information from 2020, so I'm going to asume it's 0
xunta['spain_travelers'] = np.where(xunta['spain_travelers'].isin(['..']),0,xunta['spain_travelers'])
xunta['spain_nights'] = np.where(xunta['spain_nights'].isin(['..']),0,xunta['spain_nights'])

# now converting the column into numbers
xunta['spain_travelers'] = xunta['spain_travelers'].astype(np.number)
xunta['spain_nights'] = xunta['spain_nights'].astype(np.number)

# now giving values to spain_avg_stay nights/travellers
xunta['spain_avg_stay'] = np.where(xunta['spain_avg_stay'].isin(['..']), xunta['spain_nights']/xunta['spain_travelers'],xunta['spain_avg_stay'])

# now filling NaNs that I created with 0
xunta['spain_avg_stay'] = xunta['spain_avg_stay'].fillna(0)

# now converting the column into numbers
xunta['spain_avg_stay'] = xunta['spain_avg_stay'].astype(np.number)

In [30]:
# cleanning the .. inside galicia_travelers and galicia_nights, it's the information from 2020, so I'm going to asume it's 0
xunta['galicia_travelers'] = np.where((xunta['galicia_travelers'].isin(['..'])&xunta['year'].isin([2020])),0,xunta['galicia_travelers'])
xunta['galicia_nights'] = np.where((xunta['galicia_nights'].isin(['..'])&xunta['year'].isin([2020])),0,xunta['galicia_nights'])


# cleanning the .. inside galicia_travelers and galicia_nights from the other rows, i'm going to use 0 and deal with it latter, they will be the only 0 except for covid one's so I wll have that in consideration in my latter analysis
xunta['galicia_travelers'] = np.where(xunta['galicia_travelers'].isin(['..']),0,xunta['galicia_travelers'])
xunta['galicia_nights'] = np.where(xunta['galicia_nights'].isin(['..']),0,xunta['galicia_nights'])


# now converting the column into numbers
xunta['galicia_travelers'] = xunta['galicia_travelers'].astype(np.number)
xunta['galicia_nights'] = xunta['galicia_nights'].astype(np.number)


# now giving values to galicia_avg_stay nights/travellers
xunta['galicia_avg_stay'] = np.where(xunta['galicia_travelers'].isin([0]), 0,xunta['galicia_avg_stay'])
xunta['galicia_avg_stay'] = np.where(xunta['galicia_avg_stay'].isin(['..']), xunta['galicia_nights']/xunta['galicia_travelers'],xunta['galicia_avg_stay'])


# now filling NaNs that I created with 0
xunta['galicia_avg_stay'] = xunta['galicia_avg_stay'].fillna(0)

# now converting the column into numbers
xunta['galicia_avg_stay'] = xunta['galicia_avg_stay'].astype(np.number)

In [31]:
xunta.describe()

,spain_travelers,spain_nights,spain_avg_stay,galicia_travelers,galicia_nights,galicia_avg_stay,year
count,3.120000e+03,3.120000e+03,3120.000000,3120.000000,3.120000e+03,3120.000000,3120.000000
mean,1.379117e+06,5.208234e+06,4.946570,55696.983654,1.230151e+05,2.692761,2011.338462
std,2.189060e+06,7.562056e+06,3.206214,109563.711219,2.394187e+05,1.799329,6.257758
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,2001.000000
25%,1.307880e+05,5.280475e+05,2.808056,1159.750000,3.281500e+03,1.828986,2006.000000
50%,3.845890e+05,1.747034e+06,4.100000,6540.000000,1.481550e+04,2.186577,2011.000000
75%,1.451871e+06,6.646180e+06,6.100000,34305.750000,9.503700e+04,3.101677,2017.000000
max,1.302810e+07,4.705951e+07,40.095849,726543.000000,1.615964e+06,43.000000,2022.000000


In [32]:
xunta = xunta[['accommodation_type','year','month','spain_travelers','spain_nights','spain_avg_stay','galicia_travelers','galicia_nights','galicia_avg_stay']]

In [33]:
merge3 = merge3.reset_index(drop=True)
merge3.describe()

,year,travellers_Spain,travellers_Foreign,total_travellers,nights_Spain,nights_Foreign,total_nights,avg_days
count,1143.000000,1143.000000,1143.000000,1143.000000,1.143000e+03,1143.000000,1.143000e+03,1143.000000
mean,2011.937883,60029.955381,17179.518810,77209.474191,1.382112e+05,31182.830271,1.693940e+05,2.571995
std,6.297540,105874.352828,35491.078082,138515.466222,2.408783e+05,59657.360014,2.964080e+05,1.188976
min,2001.000000,-1.000000,-1.000000,-2.000000,-1.000000e+00,-1.000000,-2.000000e+00,0.000000
25%,2006.000000,1586.500000,229.000000,2008.500000,5.267000e+03,692.500000,6.477000e+03,1.800000
50%,2012.000000,7515.000000,1203.000000,9043.000000,1.659600e+04,3000.000000,2.078700e+04,2.130000
75%,2017.000000,66669.000000,15426.500000,84263.500000,2.204635e+05,35901.000000,2.567595e+05,3.150000
max,2022.000000,573951.000000,203179.000000,726544.000000,1.357260e+06,323687.000000,1.615964e+06,9.040000


In [34]:
xunta.to_csv('xunta_per_acc.csv', index=False)
merge3.to_csv('travellers_nights_per_acc.csv', index=False)

In [35]:
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()


········


In [36]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/galicia_tourism'
engine = create_engine(connection_string)

xunta.to_sql('spain_galicia_per_acc', con = engine, if_exists = 'replace', index = False )
merge3.to_sql('trav_nights_per_acc', con = engine, if_exists = 'replace', index = False )

1143